# MQ-2 

## Ro calculation

### Specifications

#### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  can adjust
    Rh (Heater Resistance)     :  33 ohm +/- 5%
    Ph (Heating consumption)   :  less than 800mw
    

#### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 3kOhm  -  30kOhm (1000ppm iso-butane)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        LPG and Propane:  200ppm  - 5000ppm 
        Butane:           300ppm  - 5000ppm  
        Methane:          5000ppm - 20000ppm 
        H2:               300ppm  - 5000ppm 
        Alcohol:          100ppm  - 2000ppm

In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Hydrogen
h2 = {
    'p1': {'x': 200,   'y': 2.063},
    'p2': {'x': 500,   'y': 1.346},
    'p3': {'x': 800,   'y': 1.090},
    'p4': {'x': 1000,  'y': 1.000},
    'p5': {'x': 1500,  'y': 0.820},
    'p6': {'x': 2000,  'y': 0.716},
    'p7': {'x': 3000,  'y': 0.580},
    'p8': {'x': 5000,  'y': 0.458},
    'p9': {'x': 10000, 'y': 0.328}  
}

# Liquefied Petroleum Gas
lpg = {
    'p1': {'x': 200,   'y': 1.589},
    'p2': {'x': 500,   'y': 1.044},
    'p3': {'x': 800,   'y': 0.867},
    'p4': {'x': 1000,  'y': 0.771},
    'p5': {'x': 1500,  'y': 0.648},
    'p6': {'x': 2000,  'y': 0.562},
    'p7': {'x': 3000,  'y': 0.461},
    'p8': {'x': 5000,  'y': 0.358},
    'p9': {'x': 10000, 'y': 0.256}
}

# Methane
ch4 = {
    'p1': {'x': 200,   'y': 3.028},
    'p2': {'x': 500,   'y': 2.236},
    'p3': {'x': 800,   'y': 1.880},
    'p4': {'x': 1000,  'y': 1.746},
    'p5': {'x': 1500,  'y': 1.505},
    'p6': {'x': 2000,  'y': 1.346},
    'p7': {'x': 3000,  'y': 1.139},
    'p8': {'x': 5000,  'y': 0.928},
    'p9': {'x': 10000, 'y': 0.694}
}

# Carbon Monoxide
co = {
    'p1': {'x': 200,   'y': 5.125},
    'p2': {'x': 500,   'y': 3.903},
    'p3': {'x': 800,   'y': 3.343},
    'p4': {'x': 1000,  'y': 3.104},
    'p5': {'x': 1500,  'y': 2.709},
    'p6': {'x': 2000,  'y': 2.484},
    'p7': {'x': 3000,  'y': 2.154},
    'p8': {'x': 5000,  'y': 1.789},
    'p9': {'x': 10000, 'y': 1.388}
}

alcohol = {
    'p1': {'x': 200,   'y': 2.829},
    'p2': {'x': 500,   'y': 2.025},
    'p3': {'x': 800,   'y': 1.713},
    'p4': {'x': 1000,  'y': 1.581},
    'p5': {'x': 1500,  'y': 1.380},
    'p6': {'x': 2000,  'y': 1.242},
    'p7': {'x': 3000,  'y': 1.084},
    'p8': {'x': 5000,  'y': 0.873},
    'p9': {'x': 10000, 'y': 0.644}
}

smoke = {
    'p1': {'x': 200,   'y': 3.364},
    'p2': {'x': 500,   'y': 2.364},
    'p3': {'x': 800,   'y': 1.963},
    'p4': {'x': 1000,  'y': 1.823},
    'p5': {'x': 1500,  'y': 1.591},
    'p6': {'x': 2000,  'y': 1.414},
    'p7': {'x': 3000,  'y': 1.167},
    'p8': {'x': 5000,  'y': 0.923},
    'p9': {'x': 10000, 'y': 0.602}
}

#  C3H8
propane = {
    'p1': {'x': 200,   'y': 1.682},
    'p2': {'x': 500,   'y': 1.084},
    'p3': {'x': 800,   'y': 0.889},
    'p4': {'x': 1000,  'y': 0.815},
    'p5': {'x': 1500,  'y': 0.677},
    'p6': {'x': 2000,  'y': 0.591},
    'p7': {'x': 3000,  'y': 0.491},
    'p8': {'x': 5000,  'y': 0.385},
    'p9': {'x': 10000, 'y': 0.276}
}

AIR = 9.83

In [3]:
# RS value calculation
Rs = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0 = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
M = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
B = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM = lambda  racio, m, b: 10 ** ((math.log10(racio) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']

_h2      = [ h2[n]      for n in list(h2)      if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]
_smoke   = [ smoke[n]   for n in list(smoke)   if n in points]
_propane = [ propane[n] for n in list(propane) if n in points]

In [6]:
_ratios   = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

h2_ppm      = []
lpg_ppm     = []
ch4_ppm     = []
co_ppm      = []
alcohol_ppm = []
smoke_ppm   = []
propane_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _smoke)
    m = M(p1,p2)
    b = B(p1, m)
    smoke_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _propane)
    m = M(p1,p2)
    b = B(p1, m)
    propane_ppm.append(PPM(r, m, b))


#print(f"{[f'{str(i).center(6)}' for i in _ratio]}")
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','LPG', 'Propane', 'H2', 'Alcohol', 'Smoke', 'CH4', 'CO'))
for i in range(len(_ratios)):
    print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format( _ratios[i],
                   round(lpg_ppm[i],3),
                   round(propane_ppm[i],3),
                   round(h2_ppm[i],3),
                   round(alcohol_ppm[i],3),
                   round(smoke_ppm[i],3),
                   round(ch4_ppm[i],3),
                   round(co_ppm[i],3)
                  ))



Ratio    LPG           Propane       H2            Alcohol       Smoke         CH4           CO           
10       3.617         4.857         6.763         6.284         11.806        5.409         21.108       
9        4.552         6.05          8.478         8.388         15.522        7.437         30.087       
8        5.885         7.735         10.916        11.583        21.077        10.616        44.715       
7        7.875         10.219        14.538        16.701        29.815        15.894        70.071       
6        11.023        14.095        20.237        25.481        44.496        25.324        117.691      
5        16.407        20.616        29.926        41.996        71.447        43.935        217.322      
4        26.695        32.834        48.305        77.408        127.555       86.232        460.368      
3        50.0          59.829        89.553        170.286       269.284       205.694       1106.851     
2        121.086       139.373       

### Empty $V_{RL}$ Values

The minimum $V_{RL}$ voltage values within the obtained values were considered. 
The measurement took place for +/- 4 days in an empty room with the door closed (no one entered the room) and the windows open to ensure air circulation.

- __Sensor 1__: 0.17 Volts
- __Sensor 2__: 0.17 Volts

In [7]:
# MQ 2 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.17   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR  

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 2 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.17   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}"
       .format('Sensor','Rs', 'Ro','Ratio'))

print ("{:<8} {:<8} {:<8} {:<8}"
       .format(1,
               round(sensor1['rs'],3),
               round(sensor1['ro'], 3),
               round(sensor1['ratio'], 3)))

print ("{:<8} {:<8} {:<8} {:<8}"
       .format(2, 
               round(sensor2['rs'],3), 
               round(sensor2['ro'], 3), 
               round(sensor2['ratio'], 3)))

Sensor   Rs       Ro       Ratio   
1        284.118  28.903   9.83    
2        284.118  28.903   9.83    


In [8]:
## PPM
# Concentration of gases in ppm for the values of Rs/Ro (ratio) in Clean Air

sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])


sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])


sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


sensor1['smoke'] = {}
sensor1['smoke']['p1'], sensor1['smoke']['p2'] = getPoints(sensor1['ratio'], _smoke)
sensor1['smoke']['m'] = M(sensor1['smoke']['p1'], sensor1['smoke']['p2'])
sensor1['smoke']['b'] = B(sensor1['smoke']['p1'], sensor1['smoke']['m'])

sensor1['propane'] = {}
sensor1['propane']['p1'], sensor1['propane']['p2'] = getPoints(sensor1['ratio'], _propane)
sensor1['propane']['m'] = M(sensor1['propane']['p1'], sensor1['propane']['p2'])
sensor1['propane']['b'] = B(sensor1['propane']['p1'], sensor1['propane']['m'])

####

sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])


sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])


sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


sensor2['smoke'] = {}
sensor2['smoke']['p1'], sensor2['smoke']['p2'] = getPoints(sensor2['ratio'], _smoke)
sensor2['smoke']['m'] = M(sensor2['smoke']['p1'], sensor2['smoke']['p2'])
sensor2['smoke']['b'] = B(sensor2['smoke']['p1'], sensor2['smoke']['m'])

sensor2['propane'] = {}
sensor2['propane']['p1'], sensor2['propane']['p2'] = getPoints(sensor2['ratio'], _propane)
sensor2['propane']['m'] = M(sensor2['propane']['p1'], sensor2['propane']['p2'])
sensor2['propane']['b'] = B(sensor2['propane']['p1'], sensor2['propane']['m'])

####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol', 'Smoke', 'Propane'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['smoke']['m'],   sensor1['smoke']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['propane']['m'], sensor1['propane']['b']),2)
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['smoke']['m'],   sensor2['smoke']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['propane']['m'], sensor2['propane']['b']),2)
          ))

Volt     Ratio    H2            LPG           CH4           CO            Alcohol       Smoke         Propane      
0.17     9.83     7.02          3.75          5.7           22.36         6.59          12.34         5.03         
0.17     9.83     7.02          3.75          5.7           22.36         6.59          12.34         5.03         


In [9]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol', 'Smoke', 'Propane'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['smoke'] = {}
    projection['smoke']['p1'], projection['smoke']['p2'] = getPoints(r, _smoke)
    projection['smoke']['m'] = M(projection['smoke']['p1'], projection['smoke']['p2'])
    projection['smoke']['b'] = B(projection['smoke']['p1'], projection['smoke']['m'])

    projection['propane'] = {}
    projection['propane']['p1'], projection['propane']['p2'] = getPoints(r, _propane)
    projection['propane']['m'] = M(projection['propane']['p1'], projection['propane']['p2'])
    projection['propane']['b'] = B(projection['propane']['p1'], projection['propane']['m'])


    print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
               round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
               round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
               round(PPM(r, projection['co']['m'], projection['co']['b']),2),
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2),
               round(PPM(r, projection['propane']['m'], projection['propane']['b']),2)
              ))
    
    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol       Smoke         Propane      
0   0.17     9.83     7.02          3.75          5.7           22.36         6.59          12.34         5.03         
1   0.27     6.06     19.8          10.78         24.56         113.75        24.78         43.34         13.8         
2   0.37     4.33     40.76         22.46         67.89         352.75        62.31         103.85        27.84        
3   0.47     3.33     71.37         39.7          149.42        806.01        127.44        204.6         47.98        
4   0.57     2.69     113.26        63.48         286.34        1537.41       229.86        357.83        75.17        
5   0.67     2.24     168.27        94.94         500.01        2697.59       381.07        575.58        110.45       
6   0.77     1.9      238.46        135.32        776.65        4232.79       597.41        881.78        155.0        
7   0.87     1.64     326.2         186.

In [10]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4, 0.1)
RL     = 10 #kOhms
air    = 9.83  

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol', 'Smoke', 'Propane'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['smoke'] = {}
    projection['smoke']['p1'], projection['smoke']['p2'] = getPoints(r, _smoke)
    projection['smoke']['m'] = M(projection['smoke']['p1'], projection['smoke']['p2'])
    projection['smoke']['b'] = B(projection['smoke']['p1'], projection['smoke']['m'])

    projection['propane'] = {}
    projection['propane']['p1'], projection['propane']['p2'] = getPoints(r, _propane)
    projection['propane']['m'] = M(projection['propane']['p1'], projection['propane']['p2'])
    projection['propane']['b'] = B(projection['propane']['p1'], projection['propane']['m'])


    print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
               round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
               round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
               round(PPM(r, projection['co']['m'], projection['co']['b']),2),
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2),
               round(PPM(r, projection['propane']['m'], projection['propane']['b']),2)
              ))
    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol       Smoke         Propane      
0   0.17     9.83     7.02          3.75          5.7           22.36         6.59          12.34         5.03         
1   0.27     6.06     19.8          10.78         24.56         113.75        24.78         43.34         13.8         
2   0.37     4.33     40.76         22.46         67.89         352.75        62.31         103.85        27.84        
3   0.47     3.33     71.37         39.7          149.42        806.01        127.44        204.6         47.98        
4   0.57     2.69     113.26        63.48         286.34        1537.41       229.86        357.83        75.17        
5   0.67     2.24     168.27        94.94         500.01        2697.59       381.07        575.58        110.45       
6   0.77     1.9      238.46        135.32        776.65        4232.79       597.41        881.78        155.0        
7   0.87     1.64     326.2         186.

In [11]:
## Projection Ro - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/10 , sensor1['ro'] + sensor1['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol', 'Smoke', 'Propane'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])


    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])


    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])


    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])


    projectionRo['smoke'] = {}
    projectionRo['smoke']['p1'], projectionRo['smoke']['p2'] = getPoints(r, _smoke)
    projectionRo['smoke']['m'] = M(projectionRo['smoke']['p1'], projectionRo['smoke']['p2'])
    projectionRo['smoke']['b'] = B(projectionRo['smoke']['p1'], projectionRo['smoke']['m'])

    projectionRo['propane'] = {}
    projectionRo['propane']['p1'], projectionRo['propane']['p2'] = getPoints(r, _propane)
    projectionRo['propane']['m'] = M(projectionRo['propane']['p1'], projectionRo['propane']['p2'])
    projectionRo['propane']['b'] = B(projectionRo['propane']['p1'], projectionRo['propane']['m'])


    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
               round(PPM(r, projectionRo['smoke']['m'], projectionRo['smoke']['b']),2),
               round(PPM(r, projectionRo['propane']['m'], projectionRo['propane']['b']),2)
              ))

    Volt     Rs       Ro       Ratio    H2       LPG      CH4      CO       Alcohol  Smoke    Propane 
0   0.17     284.12   26.01    10.92    5.6      2.98     4.14     15.69    4.93     9.39     4.04    
1   0.17     284.12   26.11    10.88    5.64     3.01     4.19     15.89    4.99     9.48     4.07    
2   0.17     284.12   26.21    10.84    5.69     3.03     4.24     16.1     5.04     9.58     4.11    
3   0.17     284.12   26.31    10.8     5.74     3.06     4.29     16.31    5.09     9.67     4.14    
4   0.17     284.12   26.41    10.76    5.78     3.08     4.34     16.51    5.15     9.77     4.17    
5   0.17     284.12   26.51    10.72    5.83     3.11     4.39     16.73    5.2      9.86     4.2     
6   0.17     284.12   26.61    10.68    5.88     3.14     4.44     16.94    5.25     9.96     4.24    
7   0.17     284.12   26.71    10.64    5.92     3.16     4.49     17.15    5.31     10.06    4.27    
8   0.17     284.12   26.81    10.6     5.97     3.19     4.54     17.37 

In [12]:
## Projection Ro - Fixing VRL and Varying the Ro

# Sensor 2

Vcc    = 5 #volts
Vrl    = sensor2['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']
projectionRo['ro'] = np.arange(sensor2['ro'] - sensor2['ro']/10 , sensor2['ro'] + sensor2['ro']/10 , 0.1)
projectionRo['ratio'] = Ratio(projectionRo['rs'], projectionRo['ro']) 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol', 'Smoke', 'Propane'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])


    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])


    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])


    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])


    projectionRo['smoke'] = {}
    projectionRo['smoke']['p1'], projectionRo['smoke']['p2'] = getPoints(r, _smoke)
    projectionRo['smoke']['m'] = M(projectionRo['smoke']['p1'], projectionRo['smoke']['p2'])
    projectionRo['smoke']['b'] = B(projectionRo['smoke']['p1'], projectionRo['smoke']['m'])

    projectionRo['propane'] = {}
    projectionRo['propane']['p1'], projectionRo['propane']['p2'] = getPoints(r, _propane)
    projectionRo['propane']['m'] = M(projectionRo['propane']['p1'], projectionRo['propane']['p2'])
    projectionRo['propane']['b'] = B(projectionRo['propane']['p1'], projectionRo['propane']['m'])


    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
               round(PPM(r, projectionRo['smoke']['m'], projectionRo['smoke']['b']),2),
               round(PPM(r, projectionRo['propane']['m'], projectionRo['propane']['b']),2)
              ))
    
    

    Volt     Rs       Ro       Ratio    H2       LPG      CH4      CO       Alcohol  Smoke    Propane 
0   0.17     284.12   26.01    10.92    5.6      2.98     4.14     15.69    4.93     9.39     4.04    
1   0.17     284.12   26.11    10.88    5.64     3.01     4.19     15.89    4.99     9.48     4.07    
2   0.17     284.12   26.21    10.84    5.69     3.03     4.24     16.1     5.04     9.58     4.11    
3   0.17     284.12   26.31    10.8     5.74     3.06     4.29     16.31    5.09     9.67     4.14    
4   0.17     284.12   26.41    10.76    5.78     3.08     4.34     16.51    5.15     9.77     4.17    
5   0.17     284.12   26.51    10.72    5.83     3.11     4.39     16.73    5.2      9.86     4.2     
6   0.17     284.12   26.61    10.68    5.88     3.14     4.44     16.94    5.25     9.96     4.24    
7   0.17     284.12   26.71    10.64    5.92     3.16     4.49     17.15    5.31     10.06    4.27    
8   0.17     284.12   26.81    10.6     5.97     3.19     4.54     17.37 

### Ro values to be considered

- __Sensor 1__: 28.9 kOhms
- __Sensor 2__: 28.9 kOhms